<a href="https://colab.research.google.com/github/divyansh1195/Natural-Language-Processing/blob/master/SPAM%20Detection/SPAM_Detection_stem_bow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the Libraries and Dataset

In [2]:
import pandas as pd
messages = pd.read_csv('main_dataset', sep='\t', names=['LABEL', 'MESSAGES'])

messages

,LABEL,MESSAGES
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
#sns.countplot(x='LABEL', data=messages)
#plt.show()

# The dataset is an imbalanced dataset as the count of '1's are much greater than that of '0's. 

# Data Cleaning

In [5]:
import nltk
import re
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ndmke\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ndmke\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ndmke\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Stemming

Stemming may or may not give meaningful representation. However, here I am only concerned with classifying messages into spam or ham based on root form of the word. So, stemming works well here as I am not concerned with whether the root word  has a meaning or whether it is an actual word or not.

In [6]:
stemmer = PorterStemmer()
lemmatizer=WordNetLemmatizer() 
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['MESSAGES'][i])
    review = review.lower()
    review = review.split()
    review = [stemmer.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
#print(corpus)

# Create a BOW model

In [7]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 3500)
X = cv.fit_transform(corpus).toarray()

In [8]:
y = pd.get_dummies(messages['LABEL'])
y = y.iloc[:, 1].values

In [9]:
import pickle
# Creating a pickle file for the CountVectorizer
pickle.dump(cv, open('cv-transform.pkl', 'wb'))

# Train, Test Split

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Regression Model Training

**Logistic Regression**

In [19]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,y_train)
y_pred_bnb=lr.predict(X_test)

# Dump the model as a pickle

In [11]:
# Creating a pickle file for the Multinomial Naive Bayes model
filename = 'lr-model.pkl'
pickle.dump(lr, open(filename, 'wb'))